In [47]:
import pandas as pd

sales = pd.read_csv("game_sales.csv")
print(sales.head())

                       Name Platform  Year_of_Release         Genre Publisher  \
0                Wii Sports      Wii           2006.0        Sports  Nintendo   
1         Super Mario Bros.      NES           1985.0      Platform  Nintendo   
2            Mario Kart Wii      Wii           2008.0        Racing  Nintendo   
3         Wii Sports Resort      Wii           2009.0        Sports  Nintendo   
4  Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  Critic_Score  \
0     41.36     28.96      3.77         8.45         82.53          76.0   
1     29.08      3.58      6.81         0.77         40.24           NaN   
2     15.68     12.76      3.79         3.29         35.52          82.0   
3     15.61     10.93      3.28         2.95         32.77          80.0   
4     11.27      8.89     10.22         1.00         31.37           NaN   

   Critic_Count User_Score  User_Count Developer Rating 

In [48]:
ign = pd.read_excel("ign.xlsx")
print(ign.shape, ign.head())

(17534, 4)                          Game       Platform  Score                Genre
0  Wolfenstein: The New Order       Xbox One    7.8              Shooter
1                Mario Kart 8          Wii U    9.0       Racing, Action
2               Sportsfriends  PlayStation 3    8.7  Action, Compilation
3               Sportsfriends  PlayStation 4    8.7  Action, Compilation
4               Sportsfriends             PC    8.7  Action, Compilation


Now that we have those loaded into memory, it's time to see if we can combine them... We'll see how well that works

In [49]:
ign_rating = []
for game in sales['Name']:
    matches = ign[ign['Game'] == game]
    if matches.shape[0] > 0:
        ign_rating.append(matches['Score'].iloc[0])
    else:
        ign_rating.append(0)
print(ign_rating[0:10])

[7.5, 9.0, 8.5, 7.7000000000000002, 0, 9.0, 9.5, 0, 8.9000000000000004, 0]


In [50]:
# This is just a test to make sure that we can find the correct score.
print(ign[ign['Game'] == 'Super Mario Bros.'])

                   Game Platform  Score       Genre
8919  Super Mario Bros.      Wii    9.0  Platformer


Now that we've created an array, it's time to add IGN stuff to the rest of the list, then clean up the data a little bit. That means making sure the scores that are 0s in the ign column end up as NaN, not zero so we don't have any problems.

In [51]:
import numpy as np
sales['IGN'] = ign_rating
sales['IGN'] = sales['IGN'].replace(0, np.NaN)

It's time to get rid of all of those pesky NaN. First we'll find out what we're dealing with, that way we can understand how much data we're losing

In [52]:
rating_columns = ['IGN', 'User_Score', 'Critic_Score']
print(sales[pd.isnull(sales['User_Score'])].head())

                        Name Platform  Year_of_Release         Genre  \
1          Super Mario Bros.      NES           1985.0      Platform   
4   Pokemon Red/Pokemon Blue       GB           1996.0  Role-Playing   
5                     Tetris       GB           1989.0        Puzzle   
9                  Duck Hunt      NES           1984.0       Shooter   
10                Nintendogs       DS           2005.0    Simulation   

   Publisher  NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  \
1   Nintendo     29.08      3.58      6.81         0.77         40.24   
4   Nintendo     11.27      8.89     10.22         1.00         31.37   
5   Nintendo     23.20      2.26      4.22         0.58         30.26   
9   Nintendo     26.93      0.63      0.28         0.47         28.31   
10  Nintendo      9.05     10.95      1.93         2.74         24.67   

    Critic_Score  Critic_Count User_Score  User_Count Developer Rating  IGN  
1            NaN           NaN        NaN         

We might be losing a lot, but let's give it a try and see what comes of it.

In [53]:
cleaned_sales = sales
for col in rating_columns:
    cleaned_sales = cleaned_sales[pd.notnull(cleaned_sales[col])]
print(cleaned_sales.head())

                        Name Platform  Year_of_Release     Genre Publisher  \
0                 Wii Sports      Wii           2006.0    Sports  Nintendo   
2             Mario Kart Wii      Wii           2008.0    Racing  Nintendo   
3          Wii Sports Resort      Wii           2009.0    Sports  Nintendo   
6      New Super Mario Bros.       DS           2006.0  Platform  Nintendo   
8  New Super Mario Bros. Wii      Wii           2009.0  Platform  Nintendo   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  Critic_Score  \
0     41.36     28.96      3.77         8.45         82.53          76.0   
2     15.68     12.76      3.79         3.29         35.52          82.0   
3     15.61     10.93      3.28         2.95         32.77          80.0   
6     11.28      9.14      6.50         2.88         29.80          89.0   
8     14.44      6.94      4.70         2.24         28.32          87.0   

   Critic_Count User_Score  User_Count Developer Rating  IGN  
0          

While we've lost a lot of sales data, it does look a lot better. For what we want to do though the ratings are essential. So let's export this data file so we can use it to visualize the data, and then do some cool analysis.

In [54]:
cleaned_sales = cleaned_sales.reset_index()
cleaned_sales.to_csv("cleaned_game.csv")